In [3]:
from google.colab import drive
drive.mount('/content/drive')


!pip install opencv-python-headless pyswarm

import os
import cv2
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
import warnings
from pyswarm import pso

warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# GLCM Feature Extractor Class
class GLCMFeatureExtractor:
    def __init__(self):
        self.image = None
        self.tinggi = None
        self.lebar = None

    def compute_glcm_rap(self):
        GLCM0, total_piksel0 = np.zeros((256, 256)), 0
        GLCM45, total_piksel45 = np.zeros((256, 256)), 0
        GLCM90, total_piksel90 = np.zeros((256, 256)), 0
        GLCM135, total_piksel135 = np.zeros((256, 256)), 0

        for y in range(1, self.tinggi - 1):
            for x in range(1, self.lebar - 1):
                a = self.image[y, x]
                b = self.image[y, x + 1]
                GLCM0[a, b] += 1
                total_piksel0 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x + 1]
                GLCM45[a, b] += 1
                total_piksel45 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x]
                GLCM90[a, b] += 1
                total_piksel90 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x - 1]
                GLCM135[a, b] += 1
                total_piksel135 += 1

        GLCM0 /= total_piksel0
        GLCM45 /= total_piksel45
        GLCM90 /= total_piksel90
        GLCM135 /= total_piksel135

        G0 = self.compute_features(GLCM0)
        G45 = self.compute_features(GLCM45)
        G90 = self.compute_features(GLCM90)
        G135 = self.compute_features(GLCM135)

        return G0, G45, G90, G135

    def compute_features(self, GLCM):
        asm = np.sum(GLCM ** 2)
        kontras = np.sum((np.arange(256)[:, None] - np.arange(256)) ** 2 * GLCM)
        idm = np.sum(GLCM / (1 + (np.arange(256)[:, None] - np.arange(256)) ** 2))
        entropi = -np.sum(GLCM * np.where(GLCM != 0, np.log(GLCM), 0))

        px = np.sum(GLCM, axis=1)
        py = np.sum(GLCM, axis=0)
        px_py = np.outer(px, py)
        sigma_x = np.sqrt(np.sum((np.arange(256) - px) ** 2 * px))
        sigma_y = np.sqrt(np.sum((np.arange(256) - py) ** 2 * py))
        korelasi = np.sum((np.arange(256)[:, None] - px) * (np.arange(256) - py) * GLCM) / (sigma_x * sigma_y)

        features = {'asm': asm, 'kontras': kontras, 'idm': idm, 'entropi': entropi, 'korelasi': korelasi}
        return features

    def compute_glcm_features_for_channel(self, channel_image):
        self.tinggi, self.lebar = channel_image.shape
        G0, G45, G90, G135 = self.compute_glcm_rap()
        features = np.concatenate((list(G0.values()), list(G45.values()), list(G90.values()), list(G135.values())))
        return features

    def compute_glcm_features(self, image):
        self.image = image
        if len(image.shape) == 3:
            blue, green, red = cv2.split(image)
            blue_features = self.compute_glcm_features_for_channel(blue)
            green_features = self.compute_glcm_features_for_channel(green)
            red_features = self.compute_glcm_features_for_channel(red)
            main_features = np.concatenate((blue_features, green_features, red_features))
            main_features = main_features.reshape(1, -1)
        else:
            main_features = self.compute_glcm_features_for_channel(image)
        return main_features

    def print_features(self, features):
        print("Gray Level Co-occurence Matrix (GLCM)")
        print("Main features:")
        print(features)
        print("Shape Main features:")
        print(features.shape)


In [13]:
class ImageClassifier:
    def __init__(self, dataset_dir, model_dir, feature_dir, feature_type):
        self.dataset_dir = dataset_dir
        self.model_dir = model_dir
        self.feature_dir = feature_dir
        self.feature_type = feature_type
        self.data = []
        self.labels = []
        self.feature_extractors = {
            "histogram": self.extract_histogram,
            "glcm": self.extract_glcm
        }
        self.classifiers = {
            "mlp": self.train_mlp_with_pso,
            "naive_bayes": self.train_naive_bayes
        }

    def extract_histogram(self, image):
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        hist = hist.flatten()
        hist = hist.reshape(1, -1)
        return hist

    def extract_glcm(self, image):
        feature_extractor = GLCMFeatureExtractor()
        glcm_features = feature_extractor.compute_glcm_features(image)
        return glcm_features

    def load_data(self):
        for folder in os.listdir(self.dataset_dir):
            folder_path = os.path.join(self.dataset_dir, folder)

            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file)

                    if file.endswith('.jpg'):
                        image = cv2.imread(file_path)
                        features = self.feature_extractors[self.feature_type](image)

                        self.data.append(features)
                        self.labels.append(folder)

        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def train_mlp_with_pso(self):
        def objective_function(params):
            hidden_layer_size = int(params[0])
            max_iter = int(params[1])

            mlp = MLPClassifier(hidden_layer_sizes=(hidden_layer_size,), max_iter=max_iter)
            mlp.fit(self.data.reshape(len(self.data), -1), self.labels)

            y_pred = mlp.predict(self.data.reshape(len(self.data), -1))
            return -np.mean(y_pred == self.labels)  # Minimalkan negatif akurasi

        lb = [50, 100]  # Batas bawah untuk parameter (contoh: hidden_layer_size, max_iter)
        ub = [200, 500]  # Batas atas untuk parameter

        # Gunakan PSO untuk mengoptimalkan parameter
        best_params, _ = pso(objective_function, lb, ub, swarmsize=10, maxiter=10)

        # Latih klasifier MLP dengan parameter terbaik
        mlp = MLPClassifier(hidden_layer_sizes=(int(best_params[0]),), max_iter=int(best_params[1]))
        mlp.fit(self.data.reshape(len(self.data), -1), self.labels)

        return mlp

    def train_naive_bayes(self):
        nb = GaussianNB()
        nb.fit(self.data.reshape(len(self.data), -1), self.labels)
        return nb

    def train_classifier(self, classifier_type):
        X_train, X_test, y_train, y_test = train_test_split(self.data, self.labels, test_size=0.2, random_state=42)

        classifier = self.classifiers[classifier_type]()
        classifier.fit(X_train.reshape(len(X_train), -1), y_train)

        y_pred = classifier.predict(X_test.reshape(len(X_test), -1))
        print(classification_report(y_test, y_pred))

        self.classifier = classifier

    def save_classifier(self, classifier_type):
        np.save(os.path.join(self.feature_dir, 'data.npy'), self.data)
        np.save(os.path.join(self.feature_dir, 'labels.npy'), self.labels)

        classifier = self.classifier

        with open(os.path.join(self.model_dir, f'{classifier_type}_model.pkl'), 'wb') as f:
            pickle.dump(classifier, f)


if __name__ == "__main__":
    DATASET_DIR = '/content/drive/MyDrive/DATASET/backend_classification/nyobanyoba'
    MODEL_DIR = '/content/drive/MyDrive/DATASET/backend_classification/model'
    FEATURE_DIR = '/content/drive/MyDrive/DATASET/backend_classification/fitur'
    FEATURE_TYPE = 'histogram'  # pilih 'histogram' atau 'glcm'
    CLASSIFIER_TYPE = "naive_bayes"  # "mlp", "naive_bayes"

    # Buat instans ImageClassifier dan latih klasifier yang dipilih
    classifier = ImageClassifier(DATASET_DIR, MODEL_DIR, FEATURE_DIR, FEATURE_TYPE)
    classifier.load_data()
    classifier.train_classifier(CLASSIFIER_TYPE)
    classifier.save_classifier(CLASSIFIER_TYPE)

              precision    recall  f1-score   support

       Garpu       0.00      0.00      0.00         1
       Gelas       0.00      0.00      0.00         3
     Mangkok       1.00      0.50      0.67         2
      Piring       0.14      1.00      0.25         1
       Pisau       0.00      0.00      0.00         3
      Sendok       0.00      0.00      0.00         2

    accuracy                           0.17        12
   macro avg       0.19      0.25      0.15        12
weighted avg       0.18      0.17      0.13        12



In [14]:
import os
import cv2
import numpy as np
import pickle

In [15]:
class ImageClassifierTester:
    def __init__(self, model_dir, feature_dir, feature_type):
        self.model_dir = model_dir
        self.feature_dir = feature_dir
        self.feature_type = feature_type
        self.data = None
        self.labels = None
        self.classifier = None
        self.feature_extractors = {
            "histogram": self.extract_histogram,
            "glcm": self.extract_glcm
        }

    def extract_histogram(self, image):
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        hist = hist.flatten()
        # Flatten and reshape histogram to 1-dimensional array
        hist = hist.reshape(1, -1)
        return hist

    def extract_glcm(self, image):
        feature_extractor = GLCMFeatureExtractor()
        glcm_features = feature_extractor.compute_glcm_features(image)
        return glcm_features

    def load_data(self):
        self.data = np.load(os.path.join(self.feature_dir, 'data.npy'))
        self.labels = np.load(os.path.join(self.feature_dir, 'labels.npy'))

    def load_classifier(self, classifier_type):
        model_file = os.path.join(self.model_dir, f'{classifier_type}_model.pkl')
        with open(model_file, 'rb') as f:
            self.classifier = pickle.load(f)

    def read_image(self, test_image_path):
        image = cv2.imread(test_image_path)
        return image

    def process_image(self, image):
        image = image
        return image

    def test_classifier(self, test_image_path):
        image = self.read_image(test_image_path)
        image = self.process_image(image)
        features = self.feature_extractors[self.feature_type](image)
        features = features.reshape(1, -1)

        prediction = self.classifier.predict(features)
        return prediction[0], features, image


if __name__ == "__main__":
    MODEL_DIR = '/content/drive/MyDrive/DATASET/backend_classification/model'
    FEATURE_DIR = '/content/drive/MyDrive/DATASET/backend_classification/fitur'
    FEATURE_TYPE = 'histogram'  # pilih 'histogram' atau 'glcm'
    CLASSIFIER_TYPE = "naive_bayes"  # "mlp", "naive_bayes"

    TEST_IMAGE_PATH = '/content/drive/MyDrive/DATASET/backend_classification/nyobanyoba/Garpu/Garpu001.jpg'
    # Buat instans ImageClassifierTester
    tester = ImageClassifierTester(MODEL_DIR, FEATURE_DIR, FEATURE_TYPE)
    tester.load_data()
    tester.load_classifier(CLASSIFIER_TYPE)

    # Uji klasifier pada gambar uji
    prediction = tester.test_classifier(TEST_IMAGE_PATH)
    print("Prediction:", prediction)

Prediction: ('Sendok', array([[1.62351702e-03, 7.44933568e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.01113968e-05, 2.03761243e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0

In [16]:
import cv2
import numpy as np

In [17]:
class GLCMFeatureExtractor:
    def __init__(self):
        self.image = None
        self.tinggi = None
        self.lebar = None

    def compute_glcm_rap(self):
        GLCM0, total_piksel0 = np.zeros((256, 256)), 0
        GLCM45, total_piksel45 = np.zeros((256, 256)), 0
        GLCM90, total_piksel90 = np.zeros((256, 256)), 0
        GLCM135, total_piksel135 = np.zeros((256, 256)), 0

        for y in range(1, self.tinggi - 1):
            for x in range(1, self.lebar - 1):
                a = self.image[y, x]
                b = self.image[y, x + 1]
                GLCM0[a, b] += 1
                total_piksel0 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x + 1]
                GLCM45[a, b] += 1
                total_piksel45 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x]
                GLCM90[a, b] += 1
                total_piksel90 += 1

                a = self.image[y, x]
                b = self.image[y - 1, x - 1]
                GLCM135[a, b] += 1
                total_piksel135 += 1

        GLCM0 /= total_piksel0
        GLCM45 /= total_piksel45
        GLCM90 /= total_piksel90
        GLCM135 /= total_piksel135

        G0 = self.compute_features(GLCM0)
        G45 = self.compute_features(GLCM45)
        G90 = self.compute_features(GLCM90)
        G135 = self.compute_features(GLCM135)

        return G0, G45, G90, G135

    def compute_features(self, GLCM):
        asm = np.sum(GLCM ** 2)
        kontras = np.sum((np.arange(256)[:, None] - np.arange(256)) ** 2 * GLCM)
        idm = np.sum(GLCM / (1 + (np.arange(256)[:, None] - np.arange(256)) ** 2))
        entropi = -np.sum(GLCM * np.where(GLCM != 0, np.log(GLCM), 0))

        px = np.sum(GLCM, axis=1)
        py = np.sum(GLCM, axis=0)
        px_py = np.outer(px, py)
        sigma_x = np.sqrt(np.sum((np.arange(256) - px) ** 2 * px))
        sigma_y = np.sqrt(np.sum((np.arange(256) - py) ** 2 * py))
        korelasi = np.sum((np.arange(256)[:, None] - px) * (np.arange(256) - py) * GLCM) / (sigma_x * sigma_y)

        features = {'asm': asm, 'kontras': kontras, 'idm': idm, 'entropi': entropi, 'korelasi': korelasi}
        return features

    def compute_glcm_features_for_channel(self, channel_image):
        self.tinggi, self.lebar = channel_image.shape
        G0, G45, G90, G135 = self.compute_glcm_rap()
        features = np.concatenate((list(G0.values()), list(G45.values()), list(G90.values()), list(G135.values())))
        return features

    def compute_glcm_features(self, image):
        self.image = image
        if len(image.shape) == 3:
            blue, green, red = cv2.split(image)
            blue_features = self.compute_glcm_features_for_channel(blue)
            green_features = self.compute_glcm_features_for_channel(green)
            red_features = self.compute_glcm_features_for_channel(red)
            main_features = np.concatenate((blue_features, green_features, red_features))
            main_features = main_features.reshape(1, -1)
        else:
            main_features = self.compute_glcm_features_for_channel(image)
        return main_features

    def print_features(self, features):
        print("Gray Level Co-occurence Matrix (GLCM)")
        print("Main features:")
        print(features)
        print("Shape Main features:")
        print(features.shape)


if __name__ == "__main__":
    image_path = "/content/drive/MyDrive/DATASET/backend_classification/nyobanyoba/Garpu/Garpu001.jpg"
    image = cv2.imread(image_path)

    feature_extractor = GLCMFeatureExtractor()
    features = feature_extractor.compute_glcm_features(image)
    print("Main features:")
    print(features)
    print("Shape Main features:")
    print(features.shape)

Main features:
[[  0.64744025 178.25780782   0.95109651   3.49907094   0.99871618
    0.64228212  83.27176296   0.95186422   3.47404705   0.99940153
    0.64407704 236.38752761   0.94207433   3.54961682   0.99829584
    0.63728353 553.04937357   0.92191203   3.72222044   0.9960261
    0.64744025 178.25780782   0.95109651   3.49907094   0.99871618
    0.64228212  83.27176296   0.95186422   3.47404705   0.99940153
    0.64407704 236.38752761   0.94207433   3.54961682   0.99829584
    0.63728353 553.04937357   0.92191203   3.72222044   0.9960261
    0.64744025 178.25780782   0.95109651   3.49907094   0.99871618
    0.64228212  83.27176296   0.95186422   3.47404705   0.99940153
    0.64407704 236.38752761   0.94207433   3.54961682   0.99829584
    0.63728353 553.04937357   0.92191203   3.72222044   0.9960261 ]]
Shape Main features:
(1, 60)
